In [ ]:
%load_ext autoreload
%autoreload 2
from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)

from time import time_ns

from amftrack.util.dbx import upload_folders, load_dbx, download
from datetime import datetime
from amftrack.pipeline.launching.run_super import (
    run_parallel,
    directory_scratch,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer_to_archive,
)
from amftrack.util.dbx import (
    read_saved_dropbox_state,
    get_dropbox_folders_prince,
    get_dropbox_folders_prince,
    save_dropbox_state,
    get_dropbox_folders_general_recursive,
)

In [ ]:
import imageio

path = (
    r"/scratch-shared/amftrack/stitch_temp/20230723_2342_Plate20/Img3/Img_r06_c14.tif"
)
im = imageio.imread(path)

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt

plt.imshow(im)

In [ ]:
dir_drop = "DATA/PRINCE"
all_folders_drop = get_dropbox_folders_prince("/DATA/PRINCE")

In [ ]:
directory_targ = os.path.join(directory_scratch, "temp") + "/"

In [ ]:
plates = [
    "724_20220926",
    "725_20221104",
    "753_20221013",
    "757_20221019",
    "765_20221213",
    "782_20230406",
    "787_202304021",
    "793_20230317",
    "795_20230403",
    "797_20230421",
    "802_20230504",
    "812_20230404",
    "818_20220301",
    "823_20220304",
    "826_20220304",
    "832_20220419",
    "845_20220222",
    "845_20230320",
    "854_20220222",
    "859_20230401",
    "866_20220315",
    "875_20220301",
    "878_20220222",
    "878_20230501",
    "883_20220328",
    "887_2022033",
    "887_20230501",
    "889_20230504",
    "899_20230612",
    "901_20230522",
    "905_20230525",
    "916_20230530",
    "922_202305030",
    "928_20230707",
    "931_20220402",
    "933_20230623",
    "936_20230602",
    "939_20230623",
    "940_20230605",
    "947_20230706",
    "953_20220301",
    "954_20230717",
]
plates = [
    "992_20220225",
    "1001_20220304",
    "1007_20220301",
    "1016_20220509",
    "1039_20220516",
    "1063_20220705",
]
folders_drop = all_folders_drop.loc[all_folders_drop["unique_id"].isin(plates)]

In [ ]:
folders_drop

In [ ]:
plates_paper = [
    "3_20220426",
    "12_20220502",
    "16_20220419",
    "480_20221205",
]
plates_paper += [
    "52_20220707",
    "56_20220701",
    "59_20201126",
    "69_20201119",
    "76_20210108",
    "81_20220704",
    "84_20220621",
    "94_20201123",
    "102_20201226",
    "152_20201224",
    "792_20210915",
    "1076_20220511",
    "941_20220404",
]
plates_paper += [
    "1014_20220525",
    "1023_20220502",
    "1042_20220523",
    "1045_20220504",
    "1048_20220506",
    "1053_20220413",
    "127_20220705",
    "130_20220707",
    "30_20220509",
    "32_20220428",
    "33_20220502",
    "800_20210928",
]

In [ ]:
select = all_folders_drop.loc[all_folders_drop["CrossDate"] <= "20220101"]
print(len(select))
select = select.loc[~select["unique_id"].isin(plates_paper)]
print(len(select))
plates_to_move = select["unique_id"].unique()
# plates_to_move = plates_to_move[0*len(plates_to_move)//2:1*len(plates_to_move)//2]
folders_drop = all_folders_drop.loc[all_folders_drop["unique_id"].isin(plates_to_move)]
len(folders_drop)

In [ ]:
plates = os.listdir("/scratch-shared/amftrack/to_tape")
folders_drop = all_folders_drop.loc[all_folders_drop["unique_id"].isin(plates)]
folders_drop

In [ ]:
folders = folders_drop

directory_targ = os.path.join(directory_scratch, "to_tape") + "/"
directory = directory_targ
run_parallel_transfer(
    "from_drop_unflatten.py",
    [directory_targ],
    folders,
    20,
    "2:00:00",
    "staging",
    cpus=1,
    node="staging",
    name_job="archiving.sh",
)
# run_parallel_transfer_to_archive(
#     folders, directory, "10:00", "staging", dependency="archiving.sh"
# )

In [ ]:
plates_to_move

In [ ]:
folders_drop["unique_id"].unique()

In [ ]:
directory_targ = os.path.join(directory_scratch, "to_tape") + "/"
plates = list(folders_drop["unique_id"].unique())
folders = folders_drop

run_parallel_transfer_to_archive(
    folders, directory_targ, "6:00:00", "staging", dependency=None, plates=plates
)

In [ ]:
plates_to_move = plates

In [ ]:
dbx = load_dbx()

for plate in plates_to_move:
    try:
        dbx.files_delete_v2(f"/data/prince/{plate}")
    except:
        pass